In [1]:
%matplotlib inline
import sys
sys.path.append('C:/YandexDisk/Work/RevoBigScorring/')

from tempfile import TemporaryFile
import pandas as pd
import numpy as np
import os
import getpass
import pyodbc
import datetime
import matplotlib
import matplotlib.pyplot as plt

from sklearn.utils import check_consistent_length, column_or_1d, check_array
import Buckets as bc
import woe
import scoringfunctions as sf
import bsfunctions as bs
import time
import random
from scipy import stats
from math import log

from sklearn.utils.fixes import isclose
from sklearn import metrics as met
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cross_validation import KFold
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_selection import RFE
rcParams['figure.figsize'] = 7, 7
pd.options.mode.chained_assignment = None

In [2]:
connectString = 'DRIVER={SQL Server};SERVER=dwh.int.revoplus.ru;DATABASE=Revo_DW;UID=e.migaev;PWD='
pswd = getpass.getpass('DWH_Password: ')
connectString = connectString + pswd
cnxn = pyodbc.connect(connectString)
pswd = connectString = ''

DWH_Password: ········


In [3]:
mainQuery = """with bclients as (
select a.clientKey
from user_data.kuz_revo1 a
left join scorOut b on a.clientkey=b.clientkey
where apptype='2.New' and booked=1 and b.clientkey is null and year(applicationdate)>=2015 and bkiflg=1
)


select
	
	c.clientKey
	,c.guid
	,la.ApplicationDate
	,t.name as traderName
	,t.traderKey
    
    ,fa.[ApplicationId]
    ,fa.[CreditScore]
    ,fa.[CardId]
    ,fa.[FoStore]
    ,fa.[PosPolicyGroup]
    ,fa.[RegionStore]
    ,fa.[Age]
    ,fa.[CityStore]
    ,fa.[GenderTypeKey]
    ,fa.[InitialLimit]
    ,fa.[RetailerChain]
    ,fa.[RetailerPos]
    ,fa.[BkiFlg]
    ,fa.[PosBureauGroup]
    ,fa.[Segment]
    ,fa.[FinalDecision]
    ,fa.[PostBureau]
    ,fa.[PreBureau]
    ,fa.[FirstDeclineRule]
    ,fa.[Repeated]
    ,fa.[Limit]
    ,fa.[Source]
    ,fa.[Strategy]
    ,fa.[RiskGrade]
    ,fa.[ProbDef]
    ,fa.[ProbResp]
    ,fa.[MaxGapDays]
    ,fa.[MaxDifferenceInDays]
    ,fa.[TotalMaxPayment]
    ,fa.[TotalPaymentsCount]
    ,fa.[TotalPaymentsSum]
    ,fa.[LifePeriod]

	,fcr.[ApplicationId]
    ,fcr.[FirstLoanDate]
    ,fcr.[Inquiry12Month]
    ,fcr.[Inquiry1Month]
    ,fcr.[Inquiry1Week]
    ,fcr.[Inquiry3Month]
    ,fcr.[Inquiry6Month]
    ,fcr.[Inquiry9Month]
    ,fcr.[InquiryRecentPeriod]
    ,fcr.[LastLoanDate]
    ,fcr.[LoansActive]
    ,fcr.[LoansActiveMainBorrower]
    ,fcr.[LoansMainBorrower]
    ,fcr.[MaxOverdueStatus]
    ,fcr.[PayLoad]
    ,fcr.[TtlAccounts]
    ,fcr.[TtlBankruptcies]
    ,fcr.[TtlConsumer]
    ,fcr.[TtlCreditCard]
    ,fcr.[TtlDelq3059]
    ,fcr.[TtlDelq3059L12m]
    ,fcr.[TtlDelq30L12m]
    ,fcr.[TtlDelq5]
    ,fcr.[TtlDelq529]
    ,fcr.[TtlDelq6089]
    ,fcr.[TtlDelq6089L12m]
    ,fcr.[TtlDelq90Plus]
    ,fcr.[TtlDelq90PlusL12m]
    ,fcr.[TtlInquiries]
    ,fcr.[TtlLegals]
    ,fcr.[TtlOfficials]
    ,fcr.[WorstStatusEver]

from
	bclients b
		join
	client c
on
	b.clientKey = c.ClientKey
		join
	(
		select
			*
			--,Row_number() over (partition by clientKey order by ficoapplicationKey) as Farn
		from
			ficoapplication 
		where
			BkiFlg = 1
	)fa
on
	c.clientKey = fa.clientKey
		join
	loanApplication la
on
	la.loanApplicationKey = fa.loanApplicationKey
		join
	store s
on
	la.storeKey = s.storeKey
		join
	trader t
on
	t.traderKey = s.traderKey
		join
	(
		select
			*
			,Row_number() over (partition by ficoapplicationKey order by FicoCreditRegistryKey) as fcrRn
		from
			FicoCreditRegistry 
	)fcr
on
	fcr.ficoApplicationKey = fa.ficoApplicationKey
where
	fcrRn = 1 -- faRn = 1 and """

df = pd.read_sql(mainQuery,cnxn)

In [8]:
df.to_csv('addSource.csv',index=False)0

In [45]:
source = pd.read_csv('addSource.csv',encoding='cp1251')
bFlags = pd.read_csv('C:/YandexDisk/Work/Useful/BadFlags/badFlags.csv')
SourceBadFlags = pd.merge(source,bFlags,how='left')
clustInfo = pd.read_csv('clustInfo.csv')

woeOut = pd.read_csv('woeOut.csv')

In [13]:
dfVar = bs.mainTransform(SourceBadFlags)

In [15]:
dfVar.loc[dfVar.floan.isnull(),'floan'] = 0

In [37]:
dfTr = sf.ootTransform(dfVar,clustInfo,woeInfo,goodColumns)

In [40]:
dfTrCl = pd.concat([dfVar['clientKey'],dfTr],axis=1)

In [44]:
factor = 20 / log(2)
intercept = 559.50687475581992

In [46]:
dfVar['scoring'] = dfVar.apply(sf.getScoringColumn,args=[woeOut,intercept],axis=1)

In [49]:
dfVar[['clientKey','badMob3','scoring']].to_csv('scorOutL.txt',index=False)